<a href="https://colab.research.google.com/github/MensureLab/DandaraData/blob/main/DandaraData_Rev3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rapidfuzz

In [ ]:
#LABORATÓRIO: MensureLab - UFPA.
#AUTORES: @RamiroKord; @Josafha-pereira (GitHub)

import pandas as pd
import re
import unicodedata
from typing import List, Dict, Optional, Set, Tuple
from collections import defaultdict
import sys
from contextlib import redirect_stdout
import io
import csv


# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def load_communities_csv(file_path: str) -> pd.DataFrame:
    """Load communities CSV file."""
    return pd.read_csv(file_path, sep=';', encoding='latin-1')


def load_articles_csv(file_path: str) -> pd.DataFrame:
    """Load articles CSV file."""
    return pd.read_csv(file_path, sep=';', encoding='utf-8')


def get_state_name(uf: str) -> str:
    """Map UF codes to full state names."""
    UF_STATES = {
        "ac": "acre", "al": "alagoas", "am": "amazonas", "ap": "amapá",
        "ba": "bahia", "ce": "ceará", "df": "distrito federal", "es": "espírito santo",
        "go": "goiás", "ma": "maranhão", "mg": "minas gerais", "ms": "mato grosso do sul",
        "mt": "mato grosso", "pa": "pará", "pb": "paraíba", "pe": "pernambuco",
        "pi": "piauí", "pr": "paraná", "rj": "rio de janeiro", "rn": "rio grande do norte",
        "ro": "rondônia", "rr": "roraima", "rs": "rio grande do sul", "sc": "santa catarina",
        "se": "sergipe", "sp": "são paulo", "to": "tocantins"
    }
    return UF_STATES.get(uf.strip().lower(), "")


def clean_text(text: str) -> str:
    """Clean and normalize text."""
    return re.sub(r'\s+', ' ', text).strip().lower() # transforma espaços duplos, com tab, etc.. em "branco" (" ") e deixa o texto minusculo


def remove_diacritics(text: str) -> str:
    """Remove diacritical marks from text."""
    normalized = unicodedata.normalize('NFD', text) # normalized recebe a separaçãp da letra do acento, deixando no padrão NFD
    return ''.join(c for c in normalized if not unicodedata.combining(c)) # deixa apenas a parte que não tem acento


def normalize_apostrophes(text: str) -> List[str]:
    """Create variants with different apostrophe treatments."""
    if not any(char in text for char in "'`´"): # vefica se text tem algum "'", "`" ou "´". Se não tiver, então ele retorna text (nada a tratar), se não, continua
        return [text]

    apostrophe_pattern = re.compile(r'[\'`´]') #cria um padrão que corresponde aos caracteres "'", "`" ou "´"
    variants = [] # lista (agora vazia) pra armazenar versões diferentes do texto com apocrifos

    # Standard apostrophe, backtick, acute accent, space, removed
    for replacement in ["'", "`", "´", " ", ""]: # itera sobre a lista de possibilidades de substituições para os apocrifos
        variant = apostrophe_pattern.sub(replacement, text) # substitui todos os apocrifos no texto pela substituição atual de replacement e guarda em variant
                                                            # fica uma lista como por ex: "d'agua" vira ["d'agua", "dagua", "d´agua", "d agua", "dagua"] na lista variant
        if replacement == " ":
            variant = re.sub(r'\s+', ' ', variant).strip() # substitui espaço duplo, tab, etc.. por um espaço em branco.
        variants.append(variant) # vai ser adicionado em variants a lista de variant

    return variants #retorna variants


def normalize_hyphens(text: str) -> Tuple[str, str]:
    """Create hyphenated and non-hyphenated versions."""
    hyphenated = re.sub(r'\s+', '-', text) # substitui espaços, quebra de linha, etc.. por "-" -> antes: peixe boi. agr: peixe-boi
    non_hyphenated = re.sub(r'-', ' ', text) # substitui "-"" por um espaço em branco -> antes: peixe-boi. agr: peixe boi
    non_hyphenated = re.sub(r'\s+', ' ', non_hyphenated).strip() # substitui espaços duplos, tabs, etc.. em um espaço em branco normal (unico)
    return hyphenated, non_hyphenated # retorna uma dupla com hyphernated e non_hyphernated


def split_communities(text: str) -> List[str]:
    """Split community names by delimiters."""
    cleaned_text = clean_text(text) # cleaned_text recebe o texto sem espaços duplos, tabs, etc... apenas com espaços normais " " e deixa o texto minusculo

    pattern = re.compile(r'\b[eE]\b|[(),:/]') # padrão pra identificar "e" ou "E", "("", ")"", ",", ":", ou "/"

    e_count = sum(1 for m in re.finditer(r'\b[eE]\b', cleaned_text)) # conta a quantidade de vezes q "e" ou "E" (isolada) aparecem no texto

    # Special case: exactly two names separated by a single "E"
    if e_count == 1 and "," not in cleaned_text: # SE a quantidade de "e" no texto for 1 e não tiver virgula no texto limpo (cleaned_text)
        parts = [part.strip() for part in pattern.split(cleaned_text) if part.strip()] # divide o texto limpo e cria uma lista com os nomes (ex: A e B -> ["A", "B"]
        if len(parts) == 2: # se a divisão foi em duas partes... (ex: A e B -> ["A", "B"]
            return parts + [cleaned_text] # retorna parts (nome dividido) e o texto original (sem dividir). (ex: A e B -> ["A", "B", "A e B"]

    return [part.strip() for part in pattern.split(cleaned_text) if part.strip()] # Retorna a lista de comunidades dividido pelos delimitadores, com espaços e espaços extras removidos


def split_municipalities(text: str) -> List[str]:
    """Split municipality names by delimiters."""
    return [m.strip() for m in re.split(r'[|/]', text)] # retorna uma lista com o texto divido usando os delimitadoeres "|" e "/".
                                                        #  ex: São Paulo/Rio|Salvador -> ["São Paulo", "Rio", "Salvador"]


def clean_article_text(text: str) -> str:
    """Clean article text for searching."""
    cleaned_text = re.sub(r'[^\w\s]', ' ', str(text)).lower() # remove caracteres especiais e converte o texto pra minusculo
    return re.sub(r'\s+', ' ', cleaned_text).strip() # substitui espaço duplo, tab, etc.. por um espaço em branco " "


# =============================================================================
# DATA STRUCTURES
# =============================================================================

class Community:
    """Represents a quilombola community."""

    def __init__(self, id: int, name: str, municipality: str, uf: str, state: str, region: str,
                 original_municipality_str: str = None):
          '''
                           exemplo de uso:

        community = Community(id=1, name="Quilombo Kalunga", municipality="Cavalcante", uf="GO",
                    state="Goiás", region="Centro-Oeste", original_municipality_str=""Cavalcante|Goiânia")

          '''

        # para ser uma "comunidade quilombola", precisa ter esses atributos:
          self.id = id # id da comunidade
          self.name = name # nome da comunidade
          self.municipality = municipality # municipio da comunidade
          self.uf = uf # sigla da unidade federativa (ex Pará -> PA)
          self.state = state #  estado da comunidade -> convertido a partir da uf pela função get_state_name().
          self.region = region # região (ex: norte, nordeste, etc..) da comunidade
          self.original_municipality_str = original_municipality_str # municipio/Sigla-Do-Estado

    def __str__(self) -> str:
        return f"{self.name.title()}, {self.municipality.title()} ({self.uf.upper()})" #representação formatada do objeto em questão, para depuração, exibição ou relatórios
                                                                                       # ex: Quilombo Kalunga, Cavalcante (GO)


class Article:
    """Represents an academic article with search capabilities."""

    def __init__(self, id: int, title: str, title_alt1: str, title_alt2: str,
                 keywords: str, keywords_alt: str, abstract: str, abstract_alt1: str, abstract_alt2: str,
                 institution: str):
       """
                                   exemplo de uso:

                                    article = Article(
                                    id=2,
                                    title="Estudo sobre Quilombos",
                                    title_alt1="Study on Quilombos",
                                    title_alt2="Estudio sobre Quilombos",
                                    keywords="quilombo, cultura",
                                    keywords_alt="afrodescendant, culture",
                                    abstract="Comunidades quilombolas...",
                                    abstract_alt1="Quilombola communities...",
                                    abstract_alt2="Comunidades quilombolas...",
                                    institution="USP"
)
)


       """

       # para ser um artigo, precisa desses atributos:
       self.id = id # id do artigo

       # os artigos estão mostrados em 3 colunas diferentes, sendo portugues, ingles e espanhol. Porém acaba se misturando. ex: title em portugues na coluna espanhol
       self.title = title # 1 coluna de titulo
       self.title_alt1 = title_alt1 # 2 coluna de titulo
       self.title_alt2 = title_alt2 #  3 coluna de titulo

       # as palavras chave estão em 2 colunas diferentes, sendo em 2 linguas. Porém acaba se misturando igual artigos. ex: keyword em portugues na coluna espanhol
       self.keywords = keywords # 1 coluna keyword
       self.keywords_alt = keywords_alt # 2 coluna keyword

       # os resumos/abstracts estão em 3 colunas diferentes, sendo em 3 linguas. Porém acaba se misturando igual artigos. ex: abstract em portugues na coluna ingles
       self.abstract = abstract # 1 coluna de abstract
       self.abstract_alt1 = abstract_alt1 # 2 coluna de abstract
       self.abstract_alt2 = abstract_alt2 # 3 coluna de abstract
       self.institution = institution # instituição vinculada ao artigo -> universidade ou organização

        # inicializa atributos privados (pois tem o "_" apos "sef.")
        # essas variaveis guardam todas as informações do artigo em espefico (ex: artigo id 2), porém de forma concatenada.

# ==================================================================================================================================================================================
      # FUNCIONAMENTO DA LOGICA DAS VARIAVEIS ABAIXO:

        #Quando o programa faz uma busca (ex.: article.has_term_diacritical_insensitive("Quilombo Kalunga") em QuilombolaAnalyzer._check_community_match):
        #nota: suponha que está no artigo de id 2, do exemplo mais acima.

  #O método has_term_diacritical_insensitive chama get_normalized_terms_dict, que verifica:
   #se _normalized_terms_dict é None:
     #como é None, get_normalized_terms_dict chama get_normalized_full_text, que verifica:
      #se _normalized_full_text é None:
        #Como _normalized_full_text é None, ele chama get_full_text, que concatena os campos:

#_full_text vira: "Estudo sobre São Paulo Study on Sao Paulo Estudio sobre São Paulo quilombo, cultura afrodescendant, culture Comunidades quilombolas em São Paulo... Quilombola communities in Sao Paulo... Comunidades quilombolas en São Paulo...".


#get_normalized_full_text aplica remove_diacritics (tirar acentos do texto):

#_normalized_full_text vira: "Estudo sobre Sao Paulo Study on Sao Paulo Estudio sobre Sao Paulo quilombo, cultura afrodescendant, culture Comunidades quilombolas em Sao Paulo... Quilombola communities in Sao Paulo... Comunidades quilombolas en Sao Paulo...".


#get_normalized_terms_dict chama _build_terms_dict com _normalized_full_text, criando:
#python_normalized_terms_dict = {
#    "estudo": True, "sobre": True, "sao": True, "paulo": True,
#    "quilombo": True, "cultura": True, "afrodescendant": True,
#    "sao paulo": True, "sao-paulo": True, ...
#}

#has_term_diacritical_insensitive verifica se "quilombo kalunga" (normalizado para "quilombo kalunga") está em _normalized_terms_dict. Como não está, tenta variantes (ex.: com hífens) e, como última tentativa, verifica _normalized_full_text.

# =========================================================================================================================================================================================================

       self._full_text = None # tem as informações do artigo (os atributos, title, abstract, etc...), coluna por coluna, e junta, concatena tudo em uma string
       self._normalized_full_text = None # faz a mesma coisa que _full_text, porém de forma normalizada (sem acentos)
       self._terms_dict = None # cria um dicionario com "false" e "true", pra saber se a palavra está na string concatenada. com base em full_text
       self._normalized_terms_dict = None #  cria um dicionario com "false" e "true", pra saber se a palavra está na string concatenada. com base em _normalized_full_text

    def get_full_text(self) -> str:
        """Get concatenated text of all fields."""
        if self._full_text is None: # verifica se o full text é none (vazio)
            # se for verdadeiro, full_text vai receber as informações do artigo e vai concatenar todas elas
            self._full_text = " ".join([
                self.title, self.title_alt1, self.title_alt2,
                self.keywords, self.keywords_alt, self.abstract,
                self.abstract_alt1, self.abstract_alt2
            ])
        return self._full_text # retorna o texto contendo as informações do artigo em uma só string, concatenado.

    def get_normalized_full_text(self) -> str:
        """Get full text with diacritics removed."""
        if self._normalized_full_text is None: # verifica se _normalized_full_text é none (vazio)
            # se for verdadeiro, _normalized_full_text vai receber a função que remove  acentos com o argumento sendo o "ful_text".
            self._normalized_full_text = remove_diacritics(self.get_full_text()) # _normalized_full_text vai reveber o texto normalizado de full_text
        return self._normalized_full_text # retorna o texto normalizado

    def _build_terms_dict(self, text: str) -> Dict[str, bool]:
        """Build terms dictionary for fast lookup."""

        # propisito: cria um "índice" do texto de um artigo para permitir buscas rápidas de palavras ou frases que estão presentes nele.

        terms_dict = defaultdict(bool) # cria um dicionario que retorna false pra palavras inexistentes. ex: terms_dict["inexistente"], retorna False

        # Single words
        words = re.findall(r'\b\w+\b', text.lower()) # words (lista) recebe as palavras encontradas de forma individual. ex: "sao paulo e uma cidade" -> [sao, paulo, e, uma, cidade]
        for word in words: # faz um loop de word em words
            terms_dict[word] = True # se word for existente retorna true, se não existe, retorna false

        # Multi-word phrases
        phrases = re.findall(r'\b[\w\s-]+\b', text.lower()) # phrases (lista) recebe frases e palavras com espaço e hifem incluidos
        for phrase in phrases:
            phrase = phrase.strip() # strip remove cada espaço no começo e no fim  da frase
            if ' ' in phrase or '-' in phrase: # verfica se tem espaço ou hifem na frase, se tiver, continua o loop
                terms_dict[phrase] = True # se tiver, retorna true, se não (inexistente), false.

                # Add hyphen variants
                hyphenated, non_hyphenated = normalize_hyphens(phrase) # a função normalize_hyphens retorna uma dupla com 2 trocando: espaços por hifem, hifem por espaços.
                terms_dict[hyphenated] = True # ex: frase "bom dia" -> bom-dia
                terms_dict[non_hyphenated] = True # ex: frase "bom-dia" -> bom dia

                # Add apostrophe variants
                if any(char in phrase for char in "'`´"): # Se tiver qualquer caractere em "phase" com  "'`´"
                    for variant in normalize_apostrophes(phrase): # faz um loop da lista variant em normalize_apostrophes (q é uma função para fazer variantes de apostrofos)
                        terms_dict[variant] = True # se não for "inexistente" caracteres com  "'`´", então retorna true

        return terms_dict # retorna o terms_dict já pronto com os "true" e "false" de palavras que tem no texto

    def get_terms_dict(self) -> Dict[str, bool]:
        """Get terms dictionary for normal text."""
        if self._terms_dict is None: # se o _terms_dict for vazio (sem nada)
            self._terms_dict = self._build_terms_dict(self.get_full_text()) # terms_dict vai receber o resultado do texto na função que pega os termos dict (true e false) no texto
        return self._terms_dict # retorna termos dict

    def get_normalized_terms_dict(self) -> Dict[str, bool]:
        """Get terms dictionary for normalized text."""
        # Note: muitos dos codigos abaixo são quase que uma copia da f8unção de cima, só mudando ser normalizado ou não
        if self._normalized_terms_dict is None: # Se _normalized_terms_dict for vazio
            self._normalized_terms_dict = self._build_terms_dict(self.get_normalized_full_text()) # _normalized_terms_dict vai receber o resultado do texto na função que pega os termos dict (true e false) no texto
        return self._normalized_terms_dict # retorna terms dict normalizado (se acentos)

    def has_term(self, term: str) -> bool:
        """Check if term exists in article."""
        # Direct lookup
        if self.get_terms_dict().get(term, False): # verifica se existe o "term" no dicionario, caso tenha, retorns true. Caso "false", não entra na condicional.
                                                   # a.get(arg1, arg2) -> get verifica se tem o arg1 no dicionario a, se tiver, reotorna ele, se não tiver retorna o arg2
            return True # se tiver o tem no get_termos_dict, retorna true.

        # Try variants for complex terms
        if '-' in term or ' ' in term or any(char in term for char in "'`´"): # verifica se tem  em term "-" ou " ' ' " ou algum apotrofo "'`´"
            # Hyphen variants
            if '-' in term or ' ' in term: # se tiver "-" ou " ' ' " em terms, a condicional vai ser verdadeira
                hyphenated, non_hyphenated = normalize_hyphens(term) # retorna 2 versões, uma só com hifem e a outra só com espaço
                if (self.get_terms_dict().get(hyphenated, False) or
                    self.get_terms_dict().get(non_hyphenated, False)): # verificação rapida, pra saber se tem em terms_dict tems só com hifem ou só com espaço
                    return True # se sim (pegou o terms com ou sem hifem no dicionario), retorna true

            # Apostrophe variants
            if any(char in term for char in "'`´"): # se tiver algum apostrofo  "'`´" em term, a condicional vai ser verdadeira
                for variant in normalize_apostrophes(term): # faz um loop da lista variant em normalize_apostrophes (q é a função para fazer variantes de apostrofos)
                    if self.get_terms_dict().get(variant, False): # verifica se existe o "term" no dicionario, caso tenha, retorns true. Caso "false", não entra na condicional.
                                                                  # nota: no inicio da função foi explicado a logica do "get(arg1, arg2)"
                                                                  # nota 2: note que essa condição é satisfeita se tiver apostrofo em tems
                        return True # se sim (pegou alguma variação de aposcrofo de term  no dicionario), retorna true

            # Fallback to substring search
            hyphenated, non_hyphenated = normalize_hyphens(term)
            return (hyphenated in self.get_full_text() or
                   non_hyphenated in self.get_full_text()) # se nenhuma das condições acima foram satisfeitas, retorna o termo com hifem ou sem hifem no texto "completo"
                                                           # não tenho certeza se é isso, tá bem confuso esse trecho

        return False # se nenhuma condição acima for satisfeita, retorna false

    def has_term_diacritical_insensitive(self, term: str) -> bool:
        """Check if term exists ignoring diacritics."""

        # Note: muitos dos codigos abaixo são quase que uma copia da f8unção de cima, só mudando ser normalizado ou não
        normalized_term = remove_diacritics(term) # normalized_term recebe o "term" sem acentos

        # Direct lookup
        if self.get_normalized_terms_dict().get(normalized_term, False): # verifica se normalized_tem está no dicionario _normalized_terms_dict, se sim, condição feita.
                                                                         # se não, false
            return True # condição for verdadeira, retorna true

        # Try variants for complex terms
        if '-' in normalized_term or ' ' in normalized_term or any(char in normalized_term for char in "'`´"): # verifica se tem  em normalized_term "-" ou " ' ' " ou
                                                                                                               # qualquer apotrofo "'`´"
            # Hyphen variants
            if '-' in normalized_term or ' ' in normalized_term:  # se tiver "-" ou " ' ' " em normalized_term, a condicional vai ser verdadeira
                hyphenated, non_hyphenated = normalize_hyphens(normalized_term) # retorna 2 versões, uma só com hifem e a outra só com espaço
                if (self.get_normalized_terms_dict().get(hyphenated, False) or
                    self.get_normalized_terms_dict().get(non_hyphenated, False)):  # verificação rapida, pra saber se tem em terms_dict tems só com hifem ou só com espaço
                    return True  # se sim (pegou o terms com ou sem hifem no dicionario), retorna true

            # Apostrophe variants
            if any(char in normalized_term for char in "'`´"): # se tiver algum apostrofo  "'`´" em term, a condicional vai ser verdadeira
                for variant in normalize_apostrophes(normalized_term): # faz um loop da lista variant em normalize_apostrophes (q é a função para fazer variantes de apostrofos)
                    if self.get_normalized_terms_dict().get(variant, False): # verifica se existe o "term" no dicionario, caso tenha, retorns true. Caso "false", não entra na condicional.
                                                                             # nota: no inicio da função foi explicado a logica do "get(arg1, arg2)"
                                                                             # nota 2: note que essa condição é satisfeita se tiver apostrofo em tems
                        return True # se sim (pegou alguma variação de aposcrofo de term  no dicionario), retorna true

            # Fallback to substring search
            hyphenated, non_hyphenated = normalize_hyphens(normalized_term)
            return (hyphenated in self.get_normalized_full_text() or
                   non_hyphenated in self.get_normalized_full_text()) # se nenhuma das condições acima foram satisfeitas, retorna o termo com hifem ou sem hifem no texto "completo"
                                                                      # não tenho certeza se é isso, tá bem confuso esse trecho

        return False # se nenhuma condição acima for satisfeita, retorna false


class RegionalMatch:
    """Represents a regional quilombo mention."""

    def __init__(self, internal_region: str, state_name: str, uf: str, country_region: str,
                 community_term: str, all_regions_term: str = None):

        # para ser uma região, precisa desses atributos:

        self.internal_region = internal_region # região interna de um estado que a comunidade foi pesquisada. ex: sertão da bahia
        self.state_name = state_name # nome do estado
        self.uf = uf # sigla do estado
        self.country_region = country_region # região. ex: norte, nordeste, etc..
        self.community_term = community_term # termo usado no artigo para descrever as comunidades. ex: quilombos, quilombo, etc...
        self.all_regions_term = all_regions_term # usado para descrever "todas as regiões". ex: todas as regiões da bahia

    def get_community_description(self) -> str:
        """Get community description for CSV."""
        if self.internal_region == "todas as regiões": # verifica se internal region é igual a string "todas as regiões". Se for, continua pra dentro do if
            return f"COMUNIDADES DE TODAS AS REGIÕES DE {self.state_name.upper()}" # retorna "COMUNIDADES DE TODAS AS REGIÕES DE Nome-do-Estado"
        return f"COMUNIDADES DE {self.internal_region.upper()} DE {self.state_name.upper()}" # se if não foi satisfeito, vem pra cá. onde fala o nome da região interna
                                                                                             # e o nome do estado

    def get_report_text(self, article: Article) -> str:
        """Get report text for output."""
        if self.internal_region == "todas as regiões": # verifica se internal region é igual a "todas as regiões". Se sim, entra no if
            return (f"Quilombos de todas as regiões de {self.state_name.title()}, "
                   f"na região {self.country_region.title()},\n"
                   f"são mencionados no artigo de ID {article.id + 2} da tabela,\n"
                   f"estudados pela instituição {article.institution.title()}.\n") # retorna esse texto falando estado, região, id do artigo e a instituição que estudou
        return (f"Quilombos do {self.internal_region} de {self.state_name.title()}, "
               f"na região {self.country_region.title()},\n"
               f"são mencionados no artigo de ID {article.id + 2} da tabela,\n"
               f"estudados pela instituição {article.institution.title()}.\n") # Retorna esse texto falando região interna, estado,  região, id do artigo e
                                                                               # a instituição que estudou

# =============================================================================
# CORE MATCHING LOGIC
# =============================================================================

# principal classe do programa, onde serão carregados os artigos e comunidades e ocorrerá o entrelaçamento dos dados. Encontrar comunidades em artigos
class QuilombolaAnalyzer:
    """Main analyzer class with all matching logic consolidated."""

    # Constants
    # aqui ficam os nomes que são ambiguos para não confundir os algoritmos. ex: a palavra "corte" pode ser uma comunidade (q existe realmente) ou no sentido de cortar
    AMBIGUOUS_NAMES = {
        "frança", "ovo", "um", "kalunga", "base", "peixes", "forte",
        "alto", "piauí", "floresta", "américa", "brasileira",
        "araçá", "jatobá", "jurema", "aroeira", "pilões", "piloes",
        "matá", "mocambo", "solidão", "crioulo", "corte", "palmeiras",
        "porções", "porção", "batalha", "barreiras",
        "são paulo", "Pedra do Sal", "chapada", "capivari",
        "Jequitibá"
    }
    # para resolver os nomes ambiduos, antes deles devem ter algum dos seguintes termos, para ter uma "desambiguação"
    DISAMBIGUATORS = {
        "comunidade", "comunidade quilombola", "comunidade remanescente",
        "quilombo", "quilombos", "comunidade kilombola", "kilombo", "crq", "crqs",
        "território quilombola", "território kilombola", "povoado"
    }

    EXCLUDED_TERMS = {"quilombo", "quilombolas", "quilombo ", "um", "alto", "são francisco"} # termos excluidos
    PARA_STATE = "pará" # Uma exceção para deixar pará com acento, antes seria "para" poderia ser "pará" ou "para algo". Agora tem acento.

    # Regional matcher constants
    COMMUNITY_TERMS = {
        "quilombos", "comunidades", "quilombolas", "territórios", "povoados",
        "assentamentos", "kilombos", "kilombolas"
    }

    PREPOSITIONS = {"de", "da", "do", "em", "na", "no", "dos", "das"}

    INTERNAL_REGIONS = {
        "norte", "sul", "leste", "oeste", "nordeste", "sudeste", "sudoeste",
        "noroeste", "centro", "interior", "sertão", "litoral", "serra",
        "vale", "região", "regiões", "recôncavo"
    }

    ALL_REGIONS_TERMS = {
        "todas as regiões", "toda a região", "todas regiões", "toda região",
        "diversas regiões", "várias regiões", "múltiplas regiões"
    }

    STATE_NAMES = {
        "acre", "alagoas", "amazonas", "amapá", "bahia", "ceará",
        "distrito federal", "espírito santo", "goiás", "maranhão",
        "minas gerais", "mato grosso do sul", "mato grosso", "pará",
        "paraíba", "pernambuco", "piauí", "paraná", "rio de janeiro",
        "rio grande do norte", "rondônia", "roraima", "rio grande do sul",
        "santa catarina", "sergipe", "são paulo", "tocantins"
    }

    GENTILIC_TO_STATE = {
        "acreano": ("acre", "ac"), "acreana": ("acre", "ac"), "acriano": ("acre", "ac"),
        "alagoano": ("alagoas", "al"), "alagoana": ("alagoas", "al"),
        "amazonense": ("amazonas", "am"),
        "amapaense": ("amapá", "ap"),
        "baiano": ("bahia", "ba"), "baiana": ("bahia", "ba"), "baiense": ("bahia", "ba"),
        "cearense": ("ceará", "ce"),
        "brasiliense": ("distrito federal", "df"),
        "capixaba": ("espírito santo", "es"), "espírito santense": ("espírito santo", "es"),
        "goiano": ("goiás", "go"), "goiana": ("goiás", "go"),
        "maranhense": ("maranhão", "ma"),
        "mineiro": ("minas gerais", "mg"), "mineira": ("minas gerais", "mg"),
        "sul-mato-grossense": ("mato grosso do sul", "ms"),
        "mato-grossense": ("mato grosso", "mt"),
        "paraense": ("pará", "pa"), "parauara": ("pará", "pa"), "paraoara": ("pará", "pa"),
        "paraibano": ("paraíba", "pb"), "paraibana": ("paraíba", "pb"),
        "pernambucano": ("pernambuco", "pe"), "pernambucana": ("pernambuco", "pe"),
        "piauiense": ("piauí", "pi"),
        "paranaense": ("paraná", "pr"),
        "fluminense": ("rio de janeiro", "rj"),
        "potiguar": ("rio grande do norte", "rn"), "norte-rio-grandense": ("rio grande do norte", "rn"), "rio-grandense-do-norte": ("rio grande do norte", "rn"),
        "rondoniense": ("rondônia", "ro"), "rondoniano": ("rondônia", "ro"),
        "roraimense": ("roraima", "rr"),
        "gaúcho": ("rio grande do sul", "rs"), "gaúcha": ("rio grande do sul", "rs"), "sul-rio-grandense": ("rio grande do sul", "rs"), "rio-grandense-do-sul": ("rio grande do sul", "rs"),
        "catarinense": ("santa catarina", "sc"), "barriga-verde": ("santa catarina", "sc"),
        "sergipano": ("sergipe", "se"), "sergipana": ("sergipe", "se"),
        "paulista": ("são paulo", "sp"),
        "tocantinense": ("tocantins", "to")
    }

    STATE_TO_UF = {
        "acre": "ac", "alagoas": "al", "amazonas": "am", "amapá": "ap",
        "bahia": "ba", "ceará": "ce", "distrito federal": "df", "espírito santo": "es",
        "goiás": "go", "maranhão": "ma", "minas gerais": "mg", "mato grosso do sul": "ms",
        "mato grosso": "mt", "pará": "pa", "paraíba": "pb", "pernambuco": "pe",
        "piauí": "pi", "paraná": "pr", "rio de janeiro": "rj", "rio grande do norte": "rn",
        "rondônia": "ro", "roraima": "rr", "rio grande do sul": "rs", "santa catarina": "sc",
        "sergipe": "se", "são paulo": "sp", "tocantins": "to"
    }

    UF_TO_REGION = {
        "ac": "norte", "al": "nordeste", "am": "norte", "ap": "norte",
        "ba": "nordeste", "ce": "nordeste", "df": "centro-oeste", "es": "sudeste",
        "go": "centro-oeste", "ma": "nordeste", "mg": "sudeste", "ms": "centro-oeste",
        "mt": "centro-oeste", "pa": "norte", "pb": "nordeste", "pe": "nordeste",
        "pi": "nordeste", "pr": "sul", "rj": "sudeste", "rn": "nordeste",
        "ro": "norte", "rr": "norte", "rs": "sul", "sc": "sul",
        "se": "nordeste", "sp": "sudeste", "to": "norte"
    }

    # construtor para carregar tabelas de comunidade, artigo e ter os outputs.
    def __init__(self, communities_file: str, articles_file: str,
                 output_txt_file: str = "resultados.txt",
                 output_csv_file: str = "resultados_detalhados.csv",
                 articles_copy_file: str = "artigos_final.csv"):


        self.communities_file = communities_file # caminho para a tabela de comunidades quilombolas -> comunidades_quilombolas.csv
        self.articles_file = articles_file #  caminho para a tabela de artigos -> artigos_atual.csv
        self.output_txt_file = output_txt_file # caminho para a saida dos arquivos em texto -> resultados.txt
        self.output_csv_file = output_csv_file # caminho para a saida csv -> resultados_detalhados.csv
        self.articles_copy_file = articles_copy_file # Caminho para "saida final" -> artigos_final.csv

        # Load data
        self.communities = self._load_communities() # carrega a tabela de comunidades
        self.articles = self._load_articles() # carrega a tabela de artigos
        self.communities_df = load_communities_csv(communities_file) # Carrega os arquivos como DataDrame com dados brutos de comunidades (id, região, etc..)
        self.articles_df = load_articles_csv(articles_file) # Carrega os arquivos como DataDrame com dados brutos de artigos (title, abstract, etc..)

        # Results tracking
        self.article_community_matches = []  # (article_id, community_id, matched_community_name)
        self.regional_matches = []  # (article_id, RegionalMatch)
        self.csv_results = []  # (community, article) pairs for CSV export

        # Compile regional patterns
        self._compile_regional_patterns()

    def _load_communities(self) -> List[Community]:
        """Load and process communities data."""

        df = load_communities_csv(self.communities_file) # df recebe O dataFrame de comunidades quilombolas (id, região, etc..)
        communities = [] # cria uma lista (vazia) para ficar as informações da comunidade

        df['uf_clean'] = df.iloc[:, 2].apply(lambda x: x.strip().lower()) # cria a coluna "uf_clean", pega os dados da coluna 2 (UF) e rmv espaços extras e deixa em minusculo
        df['state'] = df['uf_clean'].apply(get_state_name) # cria a col "state", pega os dados de "uf_clean" e deixa com o nome do estado. ex: uf_clea: "ba" -> state: bahia

        for idx, row in df.iterrows(): # loop. idx -> indice da linha. row: pegas as informações do objeto daquela linha. ex:
                                       # idx    Nome      Idade
                                       #  0    Pessoa      20
                                       # for idx, row df.interrows -> sai
                                                                      # Nome Pessoa
                                                                      # Idade 20
            names = split_communities(clean_text(row.iloc[5])) # names recebe as comunidades divididadas
                                                               # ex: Se tem "Quilombo Kalunga, Quilombo Vão", names = ["Quilombo Kalunga", "Quilombo Vão"].
            original_muni_str = row.iloc[3].strip().lower() # original_muni_str recebe o municipio, porem sem espaços extras e em minusculo
            municipalities = split_municipalities(original_muni_str) # municipalities recebe os municipios divididos
                                                                     # ex: se tem "Cavalcante/Teresina", municipalities = ["Cavalcante", "Teresina"].

            uf = row['uf_clean'] # uf vai receber o conteudo da coluna "uf_clean"
            state = row['state'] # state vai receber o conteudo da coluna "state"
            region = row.iloc[1].strip().lower() # region recebe as regiões, porem sem espaços e minusculo

            # for em cadeia
            for name in names: # faz um loop de name em names
                for municipality in municipalities: # faz um loop de municipality em municipalities
                    community = Community(
                        id=idx, name=name, municipality=municipality,
                        uf=uf, state=state, region=region,
                        original_municipality_str=original_muni_str
                    ) # community instancia um objeto da classe Community
                    communities.append(community) # adiciona "community" a lista communities

        return communities # retorna as informações da comunidade (nome, município, estado, etc.)

    def _load_articles(self) -> List[Article]:
        """Load and process articles data."""

        df = load_articles_csv(self.articles_file) # df recebe o DataDrame com dados brutos de artigos (title, abstract, etc..)

        # Preprocess text fields
        for col in [21, 22, 23, 28, 29, 32, 33, 34, 17]: # col itera sobre as colunas listadas

            #obs: serve pra limpar o texto de todas as colunas do loop.
            col_name = f'clean_col_{col}' # col_name vai receber "clean_col_[Numero que ele está iterando]"
            df[col_name] = df.iloc[:, col].apply(clean_article_text) # texto da coluna de col_name [em relação a coluna atual do loop] é limpa

        articles = [] # cria uma lista (vazia) para colocar ficar as informações dos artigos
        for index, row in df.iterrows(): # loop. idx -> indice da linha. row: pegas as informações do objeto daquela linha. obs: tem um ex pratico mais acima
            article = Article(
                id=index,
                title=row[f'clean_col_21'],
                title_alt1=row[f'clean_col_22'],
                title_alt2=row[f'clean_col_23'],
                keywords=row[f'clean_col_28'],
                keywords_alt=row[f'clean_col_29'],
                abstract=row[f'clean_col_32'],
                abstract_alt1=row[f'clean_col_33'],
                abstract_alt2=row[f'clean_col_34'],
                institution=row[f'clean_col_17']
            )  # article instancia um objeto da classe Article
            articles.append(article) # adiciona "article" a lista articles

        return articles  # retorna as informações dos artigos

    def _compile_regional_patterns(self):
        """Compile regex patterns for regional matching."""
        community_terms = "|".join(self.COMMUNITY_TERMS) # junta community_terms em uma string separada por |. ex:community_terms = "quilombo|comunidade|mocambo
        prepositions = "|".join(self.PREPOSITIONS) #Junta preposições em uma string. ex.: prepositions = "de|do|da|das"
        internal_regions = "|".join(self.INTERNAL_REGIONS) # Junta regiões internas. ex: internal_regions = "sertão|litoral|agreste"
        state_names = "|".join(self.STATE_NAMES) # Junta nomes de estados. ex: state_names = "Bahia|Pernambuco|Pará"
        gentilics = "|".join(self.GENTILIC_TO_STATE.keys()) # Junta gentílicos ("baiano", "pernambucano"). ex: gentilics = "baiano|pernambucano"
        all_regions_terms = "|".join(self.ALL_REGIONS_TERMS) # Junta termos como "todas as regiões".ex: all_regions_terms = "todas as regiões|todas regiões"

        # Cria um padrão para até dois adjetivos opcionais, evitando confundir com preposições, regiões, ou estados.
        # ex: Para "quilombos tradicionais do sertão da Bahia", captura "tradicionais" como adjetivo.
        adjective_pattern = r'(?:\s+(?!(?:' + prepositions + r'|' + internal_regions + r'|' + state_names + r')\b)\w+){0,2}'

        # O codigo abaixo cria uma lista de quatro expressões regulares para capturar:
        # Menções com região interna e nome do estado (ex: "quilombos do sertão da Bahia")
        # Menções com região interna e gentílico (ex: "quilombos do sertão baiano")
        # Menções com "todas as regiões" e nome do estado (ex: "quilombos de todas as regiões da Bahia")
        # Menções com "todas as regiões" e gentílico (ex.: "quilombos de todas as regiões baiano")

        # Four regional patterns
        self.regional_patterns = [
            re.compile(rf'\b({community_terms}){adjective_pattern}\s+({prepositions})\s+({internal_regions})\s+({prepositions})\s+({state_names})\b', re.IGNORECASE),
            re.compile(rf'\b({community_terms}){adjective_pattern}\s+({prepositions})\s+({internal_regions})\s+({gentilics})\b', re.IGNORECASE),
            re.compile(rf'\b({community_terms}){adjective_pattern}\s+({prepositions})\s+({all_regions_terms})\s+({prepositions})\s+({state_names})\b', re.IGNORECASE),
            re.compile(rf'\b({community_terms}){adjective_pattern}\s+({prepositions})\s+({all_regions_terms})\s+({gentilics})\b', re.IGNORECASE)
        ]

    # =============================================================================
    # HIERARCHY MANAGEMENT
    #
    # Esses métodos são responsáveis por identificar relações hierárquicas entre comunidades quilombolas, ou seja, determinar se uma comunidade mencionada
    # em um artigo pode estar relacionada a outras comunidades no mesmo município ou estado, com base em seus nomes. Isso é importante porque
    # nomes de comunidades podem ser ambíguos (ex.: "Quilombo São José" pode existir em vários lugares) ou hierárquicos
    # (ex.: "Quilombo Kalunga" pode incluir subcomunidades como "Kalunga Vão").
    # =============================================================================

    def find_hierarchy_for_community(self, target_community: Community) -> List[Community]:
        """Find hierarchical relationships for a community."""

        # recebe a comunidade alvo e verifica e retorna uma lista de comunidades relacionadas, 1° por municipio e caso contrario, estado.
        # ex de problema que resolve: se o artigo menciona "São José", quais comunidades com esse nome ou variantes (ex: São José do cabo) estão no mesmo município ou estado ?

        # Step 1: Same municipality
        # IMPORTANTE: aqui no passo 1, ele verifica não só o municipio, também o estado. Mas pra ter certeza que é o municipio do estado correto. ex: tem belem no PA e PB..
        municipality_hierarchy = self._find_hierarchy_in_scope(
            target_community,
            lambda c: (c.municipality.lower() == target_community.municipality.lower() and
                      c.uf.lower() == target_community.uf.lower())
        ) # Faz um filtro que seleciona comunidades com o mesmo município (c.municipality) e estado (c.uf) da target_community, ignorando maiúsculas/minúsculas.
          # em resumo, municipality_hierarchy é uma lista de comunidades relacionadas no mesmo município da comunidade alvo.

        if len(municipality_hierarchy) > 1: # verifica se o tamanho de municipality_hierarchy (comunidades no municipio da comunidade alvo) for maior que 1
            return sorted(municipality_hierarchy, key=lambda c: len(c.name)) # retorna uma lista ordenada com o tamanho do nome, do menor pro maior

        # Se não houver mais de uma comunidade no mesmo município, então não entra no if acima, vem pra cá
        # Step 2: Same state
        state_hierarchy = self._find_hierarchy_in_scope(
            target_community,
            lambda c: c.uf.lower() == target_community.uf.lower()
        ) # verifica comunidades no mesmo estado (UF) e seleciona todas as comunidades do estado da comunidade alvo.

        if len(state_hierarchy) > 1:  # verifica se o tamanho de municipality_hierarchy (comunidades no estado da comunidade alvo) for maior que 1
            return sorted(state_hierarchy, key=lambda c: len(c.name))  # retorna uma lista ordenada com o tamanho e nome

        return [target_community] # Se nenhuma hierarquia for encontrada (no município e no estado), retorna uma lista contendo apenas a target_community.

    def _find_hierarchy_in_scope(self, target_community: Community, scope_filter) -> List[Community]:
        """Find hierarchy within a specific scope."""

        # Filtra comunidades dentro de um escopo (definido por scope_filter) e retorna aquelas que tem uma relação hierárquica com a target_community.

        scope_communities = [c for c in self.communities if scope_filter(c)] # scope_communities recebe "c" se scope_filtetr(c) for true. Logo, é uma lista de
                                                                             # comunidades que atendem ao filtro a depender dos argumentos de scope_filter

                                                                             # Scope_filter é um parametro, que quando colocado algum argumento como por exemplo:
                                                                             # c ser "Se scope_filter é lambda c: c.municipality.lower() == "cavalcante" and c.uf.lower() == "go", scope_communities",
                                                                             # ele retorna true ou false, se tem relação municipio/estado com a comunidade alvo.

        hierarchical_communities = [] # lista vazia pra retornar comunidades hierarquicamente relacionadas

        for community in scope_communities: # loop de community em scope_communities (comunidades filtradas que tem correlação com a comunidade alvo em estado e/ou municipio)
            if self._has_word_boundary_relationship(target_community, community): # Verifica quais comunidades "filtradas" tem uma relação hierarquica com a target_community
                hierarchical_communities.append(community) # se verdadeiro, vai adicionar a lista hierarchical_communities

        return hierarchical_communities # retorna a lista de comunidades hierarquicamente relacionadas com a comunidade alvo


    def _has_word_boundary_relationship(self, target_community: Community, candidate_community: Community) -> bool:
        """Check if two communities have a hierarchical relationship."""

        # Verifica se duas comunidades têm uma relação hierarquica com base em seus nomes, retornando True se forem a mesma comunidade ou
        # se um nome está contido no outro com limites de palavra. ex: "São José" está em "Quilombo São José"

        if (target_community.id == candidate_community.id and
            target_community.name == candidate_community.name): # verifica se a comunidade candidata é a comunidade alvo pelo nome e id
            return True # se for, vai retornar true

        target_variants = self._get_normalized_variants(target_community.name) # target_variants recebe diferentes variantes do nome da comunidade
        candidate_variants = self._get_normalized_variants(candidate_community.name) # candidate_variants recebe diferentes variantes do nome da comunidade

        # for em cadeia para verificar
        for target_variant in target_variants: # faz um loop de target_variant em target_variants
            for candidate_variant in candidate_variants: # faz um loop de candidate_variant em candidate_variants
                if (self._is_word_boundary_match(target_variant, candidate_variant) or
                    self._is_word_boundary_match(candidate_variant, target_variant)): # Se alguma combinação bater, retorna True
                    return True # retorna true

        return False # Se nenhuma combinação bater, retorna False

    def _get_normalized_variants(self, community_name: str) -> Set[str]:
        """Get normalized variants of a community name."""

        # faz um conjunto de variantes normalizadas de um nome de comunidade, com versões com e sem acentos e com os tratamentos de apostrofos.
        variants = set() # coleção (ou lista) não ordenada que não permite elementos duplicados

        cleaned_name = clean_text(community_name) # limpa o conteudo de community_name, retirando espaços duplos, tab, etc.. e deixando em minusculo
        variants.add(cleaned_name) # O conteudo de cleaned_name (nome limpo) é adicionado a variavel variants

        no_diacritics = remove_diacritics(cleaned_name) # no_diacritics recebe o conteudo limpo E sem acentos
        variants.add(no_diacritics) # # O conteudo de no_diacritics (conteudo limpo E sem acentos) é adicionado a variavel variants
        # obs: agora variants possui: *conteudo limpo* e *conteudo limpo E sem acentos*

        # Apostrophe variants
        apostrophe_variants = normalize_apostrophes(cleaned_name) # apostrophe_variants recebe o "cleaned_name", porém com diferentes variantes de aposcrofos
        variants.update(apostrophe_variants) # adiciona as varias versões que está em apostrophe_variants a variants

        apostrophe_variants_no_diacritics = normalize_apostrophes(no_diacritics) # apostrophe_variants_no_diacritics recebe o "no_diacritics" com diferentes variantes de aposcrofos
        variants.update(apostrophe_variants_no_diacritics)  # adiciona as varias versões que está em apostrophe_variants_no_diacritics a variants

        # obs: agora variants possui: *conteudo limpo*, *conteudo limpo E sem acentos*,
        # *conteudo limpo com diferentes variantes de aposcrofos* e  *conteudo limpo E sem acentos com diferentes variantes de aposcrofos*

        variants.discard('') # remove as strings vazias no conjunto variants
        return variants # retorna o conjunto variantes, contendo tudo oque já vimos..

    def _is_word_boundary_match(self, shorter_name: str, longer_name: str) -> bool:
        """Check if shorter name appears as complete words in longer name."""


        if shorter_name == longer_name or len(shorter_name) >= len(longer_name): # Serve pra saber se um nome A é igual ao nome B ou se o nome A é maior ou igual a B
            return False # se alguma das condições são verdadeiras, retorna falso e para.


        escaped_shorter = re.escape(shorter_name) # escaped_shorter recebe o txt de shorter_name tratado os simbulos especiais, para não ter problemas coma regex. ex: . -> \.
        pattern = r'\b' + escaped_shorter + r'\b' # pattern recebe a captura do texto com fronteira. pra não ter coisas como "bon" ser pego em "bonito"

        return bool(re.search(pattern, longer_name, re.IGNORECASE)) # verifica se pattern está em de longer_name.
                                                                    # exemplo: "SÃO PEDRO" em "SÃO PEDRO DOS BOIS", retorna TRUE.

    # =============================================================================
    # COMMUNITY MATCHING
    # =============================================================================



    def find_best_match_in_hierarchy(self, hierarchy: List[Community], article: Article) -> Optional[Community]:
        """Find the best (longest) matching community in a hierarchy."""
        # Sort by name length (longest first) for priority
        sorted_hierarchy = sorted(hierarchy, key=lambda c: len(c.name), reverse=True) # ex: "são joão" e "são jão da barra", pega são jõão da barra
                                                                                      # fica a fila: ["são jão da barra", "são joão"]

        for community in sorted_hierarchy: # faz um loop de community em sorted_hierarchy (deixa o nome mais longo como prioridade, entao o loop vai do maior nome ao menor )
            if self._check_community_match(community, article): # verifica SE a comunidade na posição "i" aparecer no artigo
                return community # se aparecer, retorna a comunidade que deu "match"

        return None # caso contrario disso, retorna nada.


    def find_best_matches_in_hierarchy(self, hierarchy: List[Community], article: Article) -> List[Community]:

        """Encontra as melhores correspondências em uma hierarquia, retornando TODAS as válidas em caso de empate."""
        sorted_hierarchy = sorted(hierarchy, key=lambda c: len(c.name), reverse=True)

        potential_matches = []
        # Encontra TODAS as comunidades na hierarquia que passam na validação.
        for community in sorted_hierarchy:
            if self._check_community_match(community, article):
                potential_matches.append(community)

        if not potential_matches:
            return []

        # Dentre as que passaram, pega o tamanho do nome da primeira (que é a mais longa).
        longest_name_len = len(potential_matches[0].name)

        # Filtra a lista para retornar APENAS as correspondências que têm esse tamanho máximo.
        final_matches = [
            match for match in potential_matches if len(match.name) == longest_name_len
        ]

        return final_matches
    def _check_community_match(self, community: Community, article: Article) -> bool:
        """Check if a community matches an article."""
        if community.name in self.EXCLUDED_TERMS: #Verifica se o nome da comunidade está em termos excluidos ("quilombo", "quilombolas")
            return False # se sim, retorna false

        # Check community name (required)
        if not article.has_term_diacritical_insensitive(community.name): # verifica se as varias versões do nome da comunidade "i" está no dicionario de palavras do artigo
            return False # se não tiver, retorna false

        # ============================ NOVA LÓGICA DE VERIFICAÇÃO ============================
        # Após confirmar que o nome curto ("Brejo") está no texto, verificamos se não existe
        # um nome hierarquicamente superior ("Brejo dos Crioulos") também presente no texto.
        # Isso evita a correspondência de sub-nomes.

        # Obtém variantes normalizadas do nome da comunidade atual para uma comparação robusta.
        current_name_variants = self._get_normalized_variants(community.name)

        # Itera sobre a lista mestra de todas as comunidades para encontrar correspondências mais específicas.
        for other_community in self.communities:
            # Otimização: Considera apenas comunidades no mesmo estado e com nomes mais longos.
            if other_community.uf == community.uf and len(other_community.name) > len(community.name):

                other_name_variants = self._get_normalized_variants(other_community.name)

                # Verifica se há uma relação hierárquica entre os nomes.
                is_hierarchical = False
                for c_variant in current_name_variants:
                    for o_variant in other_name_variants:
                        if self._is_word_boundary_match(c_variant, o_variant):
                            is_hierarchical = True
                            break
                    if is_hierarchical:
                        break

                # Se forem hierarquicamente relacionadas, verifica se o nome mais longo também está no artigo.
                if is_hierarchical and article.has_term_diacritical_insensitive(other_community.name):
                    # Se uma correspondência mais longa e específica for encontrada no texto,
                    # a atual (mais curta) é um falso positivo e deve ser descartada.
                    return False

        # Apply matching logic based on ambiguity
        if community.name in self.AMBIGUOUS_NAMES: # verifica se a comunidade está na lista de nome ambiguos
            return self._check_ambiguous_match(community, article) # se tiver, vai para a checagem especifica para nomes ambiguos
        else:
            return self._check_regular_match(community, article) # se não, vai para a checagem especifica para nomes não ambiguos

    def _check_regular_match(self, community: Community, article: Article) -> bool:
        """Check regular community matching."""
        # Check municipality first (preferred)
        if article.has_term(community.municipality): # verifica se o municipio da comunidade está em "has_term"
            return True # se sim, retorna true

        # Check state as fallback
        if community.state.lower() == self.PARA_STATE: # verifica se o estado da comunidade é o "Pará"
            return article.has_term(community.state) # se for, retorna a função has_term pra saber se "Pará" está no texto do artigo "de forma pura" (acentos)
        else:
            return article.has_term_diacritical_insensitive(community.state) # retorna a verificação do estado da comunidade, pra ver se alguma das varias versões
                                                                             # dela está no texto

    def _check_ambiguous_match(self, community: Community, article: Article) -> bool:
        """Check ambiguous community matching (requires disambiguators)."""

        ###  obs: quando essa função é chamada, garante que existe o nome da comunidade no artigo ###

        text = article.get_full_text() # text vai recebeer o texto "puro" (sem tratamento) do artigo
        mentions = 0 # menções a comunidade inicia com 0
        has_ambiguous = False

        # Find community name mentions with disambiguators
        community_pattern = r'\b' + re.escape(community.name) + r'\b' # deixa o nome da comunidade tratado pra entrr no regex. ex: "." vira "\.", para não ficar como "comando"
        occurrences = [m.start() for m in re.finditer(community_pattern, text, re.IGNORECASE)] # procura o nome da comunidade tratada no texto do artigo. insensivel a maiu/minu
                                                                                               # obs1: Se o finditer não achar correspondencia a lista fica vazia: []
                                                                                               # obs: m.start() retorna a posição do indice

        if occurrences: # se ocorrencias existir (não estiver vazio), a condição é atendida
            for pos in occurrences:
                text_before = text[:pos].strip() # text_before recebe o texto do inicio até "pos" (que é onde está o indice do nome "i")
                words_before = text_before.split()[-4:] # words_before divide o texto anterior deixa as ultimas 4 palavra antes de pós
                context = " ".join(words_before) # junta as 4 palavras em uma só string

                for disamb in self.DISAMBIGUATORS: # faz o loop de disamb em DISAMBIGUATORS (lista de palavras pra verificar se é uma comunidade, ex: quilombo, vila, etc...)
                    if re.search(r'\b' + re.escape(disamb) + r'\b', context): # procura o exato nome "i" que está em "disamb" nas 4 palavras antes do nome ambiguo
                        has_ambiguous = True
                        mentions += 1 #  se verdadeiro (ou seja,if retornar verdadeiro que tem o nome), mentions = mentions + 1
                        break # após a menção, tem uma parada e volta pro inicio do loop, até terminar os indices de occurrences


        if article.has_term(community.municipality): # Check municipality mention -> verificar se o nome do municipio aparece no artigo
            mentions += 1

        elif community.state.lower() == self.PARA_STATE: # Check state mention if municipality not mentioned -> verificar se o nome "Pará" é o nome do estado da comunidade
            if article.has_term(community.state): # verificar se o nome "Pará" aparece no artigo
                mentions += 1
        elif article.has_term_diacritical_insensitive(community.state): # verifica se o nome do estado aparece no artigo
            mentions += 1

        return has_ambiguous and mentions >= 2  # garantir que o nome realmente é uma comunidade quilombola, além das menções ao estado e municipio

    # =============================================================================
    # REGIONAL MATCHING
    # =============================================================================

    def find_regional_matches(self, article: Article) -> List[RegionalMatch]:
        """Find regional quilombo mentions in an article."""
        matches = [] # lista matches que está inicialmente vazia
        text = article.get_full_text() # text recebe o texto completlo "bruto"
        seen_combinations = set() # seen_combinations é um conjunto que não permite duplicatas nele

        #observações para pattern 1:
        #grupo 1: community_terms
        #grupo 2: prepositions
        #grupo 3: internal_regions
        #grupo 4: prepositions
        #grupo 5: state_names

        # Pattern 1: community + prep + region + prep + state , sendo "prep": preposição
        for match in self.regional_patterns[0].finditer(text): # prucura pelas correspondencia do "padrão" de regional_patterns[0] no texto. ex: quilombos do sertão da Bahia"
            community_term = match.group(1).lower() # community_term recebe o "grupo 1" (community_terms) capturado pela regex
            if not community_term.endswith('s'): # verifica se community_term termina com "s"
                continue # se verdadeiro, pula o restante do codigo e vai pra proxima iteração

            internal_region = match.group(3).lower() # internal_region recebe o grupo 3 capturado pela regex -> nome da região interna, ex: sertão
            state_name = match.group(5).lower() # state_name recebe o grupo 5 capturado pela regex -> nome do estado

            uf = self.STATE_TO_UF.get(state_name) # uf vai receber o conteudo de state_name
            if uf: # se uf não for vazio
                country_region = self.UF_TO_REGION.get(uf) # country_region vai receber a sigla do estado
                unique_key = (internal_region, state_name, uf, country_region) # unique_key vai receber as informações:
                                                                               # região interna,  nome do estado, sigla do estado, região)

                if unique_key not in seen_combinations: # se unique_key não estiver no conjunto seen_combinations, ele entra na condicional
                    seen_combinations.add(unique_key) # Adiciona o conteudo de unique_key a seen_combinations
                    matches.append(RegionalMatch(
                        internal_region=internal_region,
                        state_name=state_name,
                        uf=uf,
                        country_region=country_region,
                        community_term=community_term
                    )) # adiciona a matches o nome da região interna e o nome do estado

        #observações para pattern 2:
        #grupo 1: community_terms
        #grupo 2: prepositions
        #grupo 3: internal_regions
        #grupo 4: gentilicos

        # Pattern 2: community + prep + region + gentilic
        for match in self.regional_patterns[1].finditer(text): # prucura pelas correspondencia do "padrão" de regional_patterns[1] no texto. ex: "quilombos do sertão baiano"
            community_term = match.group(1).lower() # community_term recebe o "grupo 1" (community_terms) capturado pela regex
            if not community_term.endswith('s'):  # verifica se community_term termina com "s"
                continue # se verdadeiro, pula o restante do codigo e vai pra proxima iteração

            internal_region = match.group(3).lower() # internal_region recebe o grupo 3 capturado pela regex -> nome da região interna
            gentilic = match.group(4).lower() # gentilic recebe o grupo 4 capturado pela regex -> nome getilico. ex: baiano

            state_info = self.GENTILIC_TO_STATE.get(gentilic) # state_info vai receber o nome e sigla do estado ao qual o gentilico se refere. ex|: baiano -> [Bahia e BA]
            if state_info: # se state_info tiver algum conteudo nele
                state_name, uf = state_info # state_name recebe o nome do estado que está em state_info e uf recebe a sigla do estado que está em state_info
                country_region = self.UF_TO_REGION.get(uf) # country_region vai receber a região onde se encontra o estado
                unique_key = (internal_region, state_name, uf, country_region) # unique_key vai receber as informações:
                                                                               # região interna,  nome do estado, sigla do estado, região)

                if unique_key not in seen_combinations: # se unique_key não estiver no conjunto seen_combinations, ele entra na condicional
                    seen_combinations.add(unique_key) # Adiciona o conteudo de unique_key a seen_combinations
                    matches.append(RegionalMatch(
                        internal_region=internal_region,
                        state_name=state_name,
                        uf=uf,
                        country_region=country_region,
                        community_term=community_term
                    )) # adiciona a matches o nome da região interna e o nome do estado


        #observações para pattern 3:
        #grupo 1: community_terms
        #grupo 2: prepositions
        #grupo 3: all_regions_terms
        #grupo 4: prepositions
        #grupo 5: state_names

        # Pattern 3: community + prep + "todas as regiões" + prep + state
        for match in self.regional_patterns[2].finditer(text):  # procura pelas correspondencia do "padrão" de regional_patterns[2] no texto. ex: quilombos de todas as regiões da Bahia
            community_term = match.group(1).lower() # community_term recebe o "grupo 1" (community_terms) capturado pela regex
            if not community_term.endswith('s'): # verifica se community_term termina com "s"
                continue # se verdadeiro, pula o restante do codigo e vai pra proxima iteração

            all_regions_term = match.group(3).lower() # all_regions_term recebe o grupo 3 capturado pela regex -> ex: todas as regiões
            state_name = match.group(5).lower() # state_name vai receber o nome do estado

            uf = self.STATE_TO_UF.get(state_name) # uf recebe a sigla do estado
            if uf:
                country_region = self.UF_TO_REGION.get(uf) # country_region recebe o nome da região onde se encontra o estado
                internal_region = "todas as regiões" # internal_region recebe a string  "todas as regiões"
                unique_key = (internal_region, state_name, uf, country_region) # unique_key vai receber as informações:
                                                                               # região interna,  nome do estado, sigla do estado, região)

                if unique_key not in seen_combinations: # se unique_key não estiver no conjunto seen_combinations, ele entra na condicional
                    seen_combinations.add(unique_key) # Adiciona o conteudo de unique_key a seen_combination
                    matches.append(RegionalMatch(
                        internal_region=internal_region,
                        state_name=state_name,
                        uf=uf,
                        country_region=country_region,
                        community_term=community_term,
                        all_regions_term=all_regions_term
                    )) # adiciona a matches "COMUNIDADES DE TODAS AS REGIÕES DE Nome-do-Estado"


        #observações para pattern 3:
        #grupo 1: community_terms
        #grupo 2: prepositions
        #grupo 3: all_regions_terms
        #grupo 4: gentilics

        # Pattern 4: community + prep + "todas as regiões" + gentilic
        for match in self.regional_patterns[3].finditer(text): # procura pelas correspondencia do "padrão" de regional_patterns[3] no texto. ex: quilombos de todas as regiões baiano
            community_term = match.group(1).lower() # community_term recebe o "grupo 1" (community_terms) capturado pela regex
            if not community_term.endswith('s'): # verifica se community_term termina com "s"
                continue # se verdadeiro, pula o restante do codigo e vai pra proxima iteração

            all_regions_term = match.group(3).lower() # all_regions_term recebe o grupo 3 capturado pela regex -> ex: todas as regiões
            gentilic = match.group(4).lower()  # gentilic recebe o grupo 4 capturado pela regex -> nome getilico. ex: baiano

            state_info = self.GENTILIC_TO_STATE.get(gentilic) # state_info vai receber o nome e sigla do estado ao qual o gentilico se refere. ex|: baiano -> [Bahia e BA]
            if state_info: # se state_info tiver algum conteudo nele
                state_name, uf = state_info # state_name recebe o nome do estado que está em state_info e uf recebe a sigla do estado que está em state_info
                country_region = self.UF_TO_REGION.get(uf)  # country_region vai receber a região onde se encontra o estado
                internal_region = "todas as regiões" # internal_region recebe a string  "todas as regiões"
                unique_key = (internal_region, state_name, uf, country_region) # unique_key vai receber as informações:
                                                                               # região interna,  nome do estado, sigla do estado, região)

                if unique_key not in seen_combinations: # se unique_key não estiver no conjunto seen_combinations, ele entra na condicional
                    seen_combinations.add(unique_key) # o conteudo de unique_key é adicionado a seen_combinations
                    matches.append(RegionalMatch(
                        internal_region=internal_region,
                        state_name=state_name,
                        uf=uf,
                        country_region=country_region,
                        community_term=community_term,
                        all_regions_term=all_regions_term
                    )) # adiciona a matches "COMUNIDADES DE TODAS AS REGIÕES DE Nome-do-Estado"

        return matches # retorna o conteudi de matches

    # =============================================================================
    # REPORTING
    # =============================================================================

    def generate_community_report(self, community: Community, article: Article) -> str:
        """Generate match report for a community."""
        prefix = "Nome ambíguo" if community.name in self.AMBIGUOUS_NAMES else "Comunidade"

        # CONDIÇÃO CORRIGIDA: A única condição para mostrar a lista completa
        # de municípios é se a comunidade, na sua origem, pertencer a múltiplos municípios.
        if community.original_municipality_str and "|" in community.original_municipality_str:
            # Lógica para agrupar e formatar múltiplos municípios
            all_municipalities = split_municipalities(community.original_municipality_str)

            muni_parts = [f"{m.title()} ({community.uf.upper()})" for m in all_municipalities[:-1]]
            muni_str = ", ".join(muni_parts)
            muni_str += f", e/ou {all_municipalities[-1].title()} ({community.uf.upper()})"

            return (f"{prefix} {community.name.title()}, do município {muni_str}, "
                    f"da região {community.region.upper()},\n"
                    f"é mencionada no artigo de ID {article.id + 2} da tabela,\n"
                    f"estudada pela instituição {article.institution.title()}.\n")
        else:
            # Relatório padrão para comunidades com um único município
            return (f"{prefix} {community.name.title()}, do município {community.municipality.title()} "
                    f"({community.uf.upper()}), da região {community.region.upper()},\n"
                    f"é mencionada no artigo de ID {article.id + 2} da tabela,\n"
                    f"estudada pela instituição {article.institution.title()}.\n")

    # =============================================================================
    # EXPORT FUNCTIONS
    # =============================================================================

    def export_csv(self):
        """Export detailed results to CSV."""
        headers = [
            "COMUNIDADE", "MUNICÍPIO", "UF", "REGIÃO", "AUTORES",
            "TÍTULO DO ARTIGO (IDIOMA 1)", "TÍTULO DO ARTIGO (IDIOMA 2)",
            "TÍTULO DO ARTIGO (IDIOMA 3)", "REVISTA", "PALAVRAS-CHAVE (IDIOMA 1)",
            "PALAVRAS-CHAVE (IDIOMA 2)", "RESUMO (IDIOMA 1)", "RESUMO (IDIOMA 2)",
            "RESUMO (IDIOMA 3)", "INSTITUIÇÃO"
        ]

        with open(self.output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile, delimiter=';')
            writer.writerow(headers)

            for community, article in self.csv_results:
                row = [
                    community.name.title(),
                    community.municipality.title(),
                    community.uf.upper(),
                    community.region.upper(),
                    "",  # AUTORES
                    article.title,
                    article.title_alt1,
                    article.title_alt2,
                    "",  # REVISTA
                    article.keywords,
                    article.keywords_alt,
                    article.abstract,
                    article.abstract_alt1,
                    article.abstract_alt2,
                    article.institution.title()
                ]
                writer.writerow(row)

        return len(self.csv_results)

    def create_enhanced_articles_copy(self):
        """Create enhanced articles copy with community data."""
        copy_df = self.articles_df.copy()
        all_rows = []

        # Add rows for community matches
        for article_id, community_id, matched_community_name in self.article_community_matches:
            new_row = self.articles_df.iloc[article_id].copy()
            community_row = self.communities_df.iloc[community_id]

            # Copy first 17 columns from community
            for col_idx in range(min(17, len(community_row))):
                new_row.iloc[col_idx] = community_row.iloc[col_idx]

            # Replace community name with matched name
            new_row.iloc[5] = matched_community_name.upper()
            all_rows.append(new_row)

        # Add rows for regional matches
        for article_id, regional_match in self.regional_matches:
            new_row = self.articles_df.iloc[article_id].copy()

            new_row.iloc[0] = ""  # Empty cell
            new_row.iloc[1] = regional_match.country_region.upper()
            new_row.iloc[2] = regional_match.uf.upper()
            new_row.iloc[3] = ""  # Municipality
            new_row.iloc[4] = ""  # Other field
            new_row.iloc[5] = regional_match.get_community_description()

            all_rows.append(new_row)

        # Create final dataframe
        matched_df = pd.DataFrame(all_rows, columns=copy_df.columns)

        # Get unmatched articles
        matched_article_ids = set(aid for aid, _, _ in self.article_community_matches)
        matched_article_ids.update(aid for aid, _ in self.regional_matches)

        unmatched_df = copy_df.iloc[[i for i in range(len(copy_df)) if i not in matched_article_ids]]

        # Combine and save
        final_df = pd.concat([unmatched_df, matched_df], ignore_index=True)
        final_df.to_csv(self.articles_copy_file, sep=';', encoding='utf-8', index=False)

        return len(self.article_community_matches), len(self.regional_matches), len(unmatched_df)

    def add_year_column(self):
        """Extract year from column 13 and add to last column."""
        df = pd.read_csv(self.articles_copy_file, sep=';', encoding='utf-8')

        last_four_chars = df.iloc[:, 13].astype(str).apply(
            lambda x: x[-4:] if len(x) >= 4 else x
        )

        df['ANO DA PORTARIA'] = last_four_chars
        df.to_csv(self.articles_copy_file, sep=';', encoding='utf-8', index=False)

        return len(df)

    # =============================================================================
    # MAIN ANALYSIS
    # =============================================================================

    def analyze(self) -> int:
        """Função de análise principal com priorização geográfica e prevenção de duplicatas."""
        output_buffer = io.StringIO()
        final_matches_per_article = defaultdict(list)

        # Passos 1-5: Lógica da "versão quase-final-2.0" para encontrar os melhores candidatos
        for article in self.articles:
            level_2_candidates, level_1_candidates, level_0_candidates = [], [], []
            for community in self.communities:
                if article.has_term_diacritical_insensitive(community.name):
                    municipality_found = article.has_term_diacritical_insensitive(community.municipality)
                    state_found = article.has_term_diacritical_insensitive(community.state)
                    if municipality_found and state_found: level_2_candidates.append(community)
                    elif municipality_found: level_1_candidates.append(community)
                    elif state_found: level_0_candidates.append(community)

            chosen_candidates = []
            if level_2_candidates: chosen_candidates = level_2_candidates
            elif level_1_candidates: chosen_candidates = level_1_candidates
            elif level_0_candidates: chosen_candidates = level_0_candidates

            if not chosen_candidates: continue

            final_valid_matches = []
            for candidate in chosen_candidates:
                if self._check_community_match(candidate, article):
                    final_valid_matches.append(candidate)

            unique_matches = []
            seen_matches = set()
            for match in final_valid_matches:
                match_key = (match.id, match.name, match.municipality, match.uf)
                if match_key not in seen_matches:
                    unique_matches.append(match)
                    seen_matches.add(match_key)

            if unique_matches:
                final_matches_per_article[article.id] = unique_matches

        # PASSO 6: GERAÇÃO DE RELATÓRIOS COM LÓGICA DE CONSOLIDAÇÃO POR ID
        matches_count = 0
        with redirect_stdout(output_buffer):
            print("=== PRIMEIRA RODADA: COMUNIDADES ESPECÍFICAS ===\n")
            article_ids_with_matches = sorted(final_matches_per_article.keys())

            for article_id in article_ids_with_matches:
                article = self.articles[article_id]

                # --- LÓGICA DE CONSOLIDAÇÃO ADICIONADA AQUI ---
                # Agrupa os resultados pelo ID da linha original do CSV.
                grouped_by_id = defaultdict(list)
                for match in final_matches_per_article[article_id]:
                    grouped_by_id[match.id].append(match)

                # Ordena os grupos pelo nome da comunidade para uma saída consistente.
                sorted_groups = sorted(grouped_by_id.values(), key=lambda group: group[0].name)

                # Itera sobre os grupos consolidados. Cada "grupo" representa uma única comunidade real.
                for group in sorted_groups:
                    # Pega o primeiro item do grupo como representante.
                    representative_match = group[0]

                    # Gera o relatório de texto UMA VEZ por grupo, usando a função corrigida.
                    report = self.generate_community_report(representative_match, article)
                    print(report)

                    # Cria uma representação consolidada para o arquivo CSV.
                    all_munis_str = representative_match.original_municipality_str.replace('|', ', ').title()
                    csv_representation = Community(
                        id=representative_match.id, name=representative_match.name,
                        municipality=all_munis_str,
                        uf=representative_match.uf, state=representative_match.state, region=representative_match.region,
                        original_municipality_str=representative_match.original_municipality_str
                    )

                    # Adiciona aos resultados para exportação UMA VEZ por grupo.
                    self.csv_results.append((csv_representation, article))
                    self.article_community_matches.append((article.id, representative_match.id, representative_match.name))
                    matches_count += 1

            # O resto da função para gerar o resumo e exportar os arquivos permanece o mesmo.
            print("=== SEGUNDA RODADA: MENÇÕES REGIONAIS ===\n")
            unique_regional_articles = set()
            for article in self.articles:
                if article.id not in final_matches_per_article:
                    regional_match_list = self.find_regional_matches(article)
                    if regional_match_list:
                        unique_regional_articles.add(article.id)
                        for regional_match in regional_match_list:
                            report = regional_match.get_report_text(article)
                            print(report)
                            self.regional_matches.append((article.id, regional_match))
            regional_matches_count = len(unique_regional_articles)

            print("=== RESUMO DOS RESULTADOS ===\n")
            unique_article_count = len(final_matches_per_article)
            print(f"Foram encontradas {matches_count} menções a comunidades quilombolas específicas em {unique_article_count} artigos únicos.")
            print(f"Foram identificados {regional_matches_count} artigos adicionais que mencionam quilombos/comunidades de forma regional.")
            total_articles = unique_article_count + regional_matches_count
            print(f"Total de artigos com menções relevantes: {total_articles}")

        with open(self.output_txt_file, 'w', encoding='utf-8') as f:
            f.write(output_buffer.getvalue())
        print(output_buffer.getvalue())
        csv_records = self.export_csv()
        print(f"CSV com {csv_records} registros detalhados exportado para: {self.output_csv_file}")
        first_count, second_count, unmatched_count = self.create_enhanced_articles_copy()
        print(f"Nova cópia dos artigos salva em: {self.articles_copy_file}")
        print(f"  - {first_count} linhas para artigos com comunidades específicas")
        print(f"  - {second_count} linhas para artigos com menções regionais")
        print(f"  - {unmatched_count} linhas para artigos sem comunidades")
        self.add_year_column()
        print(f"Processamento concluído. Últimos quatro caracteres da coluna 13 adicionados\n"
              f"como nova coluna 'ANO DA PORTARIA' em: {self.articles_copy_file}")

        return total_articles


# =============================================================================
# MAIN FUNCTION
# =============================================================================

def main():
    """Main function for running the analysis."""
    # Mount Google Drive (specific to Google Colab)
    try:
        from google.colab import drive
        drive.mount('/content/drive/')

        # File paths for Colab
        communities_file = '/content/drive/My Drive/estudo_quilombos_organizado/crqs_atual.csv'
        articles_file = '/content/drive/My Drive/estudo_quilombos_organizado/artigos_atual.csv'
        output_txt_file = '/content/drive/My Drive/estudo_quilombos_organizado/resultados.txt'
        output_csv_file = '/content/drive/My Drive/estudo_quilombos_organizado/resultados_detalhados.csv'
        articles_copy_file = '/content/drive/My Drive/estudo_quilombos_organizado/artigos_final.csv'
    except ImportError:
        # Local file paths (fallback)
        communities_file = 'crqs_atual.csv'
        articles_file = 'artigos_atual.csv'
        output_txt_file = 'resultados.txt'
        output_csv_file = 'resultados_detalhados.csv'
        articles_copy_file = 'artigos_final.csv'

    # Run analysis
    analyzer = QuilombolaAnalyzer(
        communities_file=communities_file,
        articles_file=articles_file,
        output_txt_file=output_txt_file,
        output_csv_file=output_csv_file,
        articles_copy_file=articles_copy_file
    )

    total_matches = analyzer.analyze()

    print(f"\nAnálise concluída com {total_matches} correspondências encontradas.")
    print(f"Resultados salvos em:")
    print(f"  - Texto: {output_txt_file}")
    print(f"  - CSV detalhado: {output_csv_file}")
    print(f"  - Artigos com comunidades: {articles_copy_file}")


if __name__ == "__main__":
    main()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
=== PRIMEIRA RODADA: COMUNIDADES ESPECÍFICAS ===

Comunidade Ilhotinha, do município Capivari De Baixo (SC), da região SUL,
é mencionada no artigo de ID 3 da tabela,
estudada pela instituição N D.

Comunidade Córrego Do Rocha, do município Chapada Do Norte (MG), da região SUDESTE,
é mencionada no artigo de ID 5 da tabela,
estudada pela instituição N D.

Comunidade Lage Dos Negros, do município Campo Formoso (BA), da região NORDESTE,
é mencionada no artigo de ID 7 da tabela,
estudada pela instituição N D.

Comunidade Caraíbas, do município Pedras De Maria Da Cruz (MG), da região SUDESTE,
é mencionada no artigo de ID 8 da tabela,
estudada pela instituição N D.

Comunidade Caraíbas, do município Cônego Marinho (MG), da região SUDESTE,
é mencionada no artigo de ID 8 da tabela,
estudada pela instituição N D.

Comunidade Imbiral Cabeça Branca, do município Pedro 